# Clasificación multiclase
##### Nombre: Soria Colque Franz Ronald
 
github: https://github.com/FranzDS01/InteligenciaArtifical/blob/main/Examen%20Laboratorio/Clasificacion_multiclase.ipynb

 Alumno: Soria Colque Franz Ronald
 Carrera: Ingenieria de Sistemas
 Grupo laboratorio: Miercoles
 Materia: sis420

Dataset :https://www.kaggle.com/datasets/uciml/glass

Calsificacion de tipo de vidrio 
Columnas:
RI: índice de refracción
Na: Sodio (unidad de medida: porcentaje en peso en óxido correspondiente, como son los atributos 4-10)
Mg: Magnesio
Al: Aluminio
Si: Silicio
K: Potasio
Ca: Calcio
Ba: Bario
Fe: Hierro
Tipo de vidrio:6 tipos

Mejor valor de lamnda=0.002
Mejor valor de maxIter=1500
Mejor valor de theta :
[[-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [ 1.89656186e+01  1.93975898e-01 -1.12016860e-01  1.86676605e-01
  -3.50816074e-01 -2.42674154e-01 -1.25760901e-01 -8.02184625e-02
  -4.52411897e-01 -1.09966833e+00]
 [-1.61651377e+00 -4.27264539e-02 -3.63512451e-01  9.03462918e-02
   3.25894494e-02  6.16630311e-02  3.28827025e-03  1.09738519e-01
   9.81226155e-02  2.75455887e+00]
 [ 2.24467555e+01 -6.63300752e-01 -2.02616906e-01  1.19692581e-01
  -2.97595063e-01 -2.87098144e-01  1.94180693e-02 -2.74849196e-02
   1.08209975e-01 -9.01805344e-01]
 [-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [-2.10186903e+01  9.10684499e-01  1.75128729e-01 -1.38587569e-01
   7.06298698e-01  1.69360145e-01  3.97495922e-01  1.41021682e-01
  -6.12505514e-01  1.23209096e+00]]





In [866]:

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
%matplotlib inline

Verfficacion de cantidad de clasificaciones que tiene la columna y ="type"

In [867]:
data =pd.read_csv('./dataset/glass.csv')
value = list(data["Type"].unique())
print("numero de clases",value)
data.info()

numero de clases [1, 2, 3, 5, 6, 7]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


#funcion DatosSinteticos(data) recibe como parametro el dataset
apartir de eso crear datos sinteticos n_filas=define la cantidad de datos que se va crear

Luego crea una matriz numpy vacía con la misma cantidad de filas y columnas que el dataset oiginal

Luego de con el for se itera cada columna y toma datos aleatorios de es a columna que sera el nuevo dato sintetico.[muestras_i]
luego esa [muestras_i] generada se agrega [nuevos_datos] en la fila y columna correspondiente

Una ves que se termina de iterar , se crear un [DataFrame] nuevo de los datos sinteticos
y [dataExtent] se concantena los datos originales con los sinteticos
retornando[dataExtent] como el nuevo dataset

In [868]:
def DatosSinteticos(data):
    n_filas=10000
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    for i in range(len(data.columns)):
        columna_i =data.iloc[:,i]
        muestras_i=np.random.choice(columna_i,n_filas,replace=True)
        nuevos_datos[:,i]=muestras_i.round()
    columnas =list(data.columns)
    nuevos_datos_df=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_df],axis=0)
    #print(dataExtend.tail())
    return dataExtend

In [869]:
data=DatosSinteticos(data)

Verificacion del nuevo dataset ahora tiene mas de [10000] datos

In [870]:
value = list(data["Type"].unique())
print(len(value))
data.info()

6
<class 'pandas.core.frame.DataFrame'>
Index: 10214 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      10214 non-null  float64
 1   Na      10214 non-null  float64
 2   Mg      10214 non-null  float64
 3   Al      10214 non-null  float64
 4   Si      10214 non-null  float64
 5   K       10214 non-null  float64
 6   Ca      10214 non-null  float64
 7   Ba      10214 non-null  float64
 8   Fe      10214 non-null  float64
 9   Type    10214 non-null  float64
dtypes: float64(10)
memory usage: 877.8 KB


In [871]:
data=data.to_numpy()

[num_labels] que para este caso es [6] tipos de clasificacion 
[X] contiene los primeros [500] filas del dataset para entrenamiento
[y] de la misma forma [500]

y para datos de prueba se esta tomando a partir de la fila [500] en [Xprueba yprueba]

In [872]:
num_labels = 6
X, y = data[0:500, :9], data[0:500, 9]
Xprueba=data[500:,:9]
yprueba=data[500:,9]
m = y.size
print(X.shape)

(500, 9)


In [873]:
print(X)
print(y)

[[ 1.52101 13.64     4.49    ...  8.75     0.       0.     ]
 [ 1.51761 13.89     3.6     ...  7.83     0.       0.     ]
 [ 1.51618 13.53     3.55    ...  7.78     0.       0.     ]
 ...
 [ 2.      13.       4.      ...  8.       0.       0.     ]
 [ 2.      15.       0.      ...  8.       0.       0.     ]
 [ 2.      14.       4.      ...  9.       0.       0.     ]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 6. 6. 6. 6. 6. 6. 6. 6. 6. 7. 7. 7. 7. 7. 7. 7.
 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7

### 1.3 Vectorización de regresión logística

Se utilizará múltiples modelos de regresión logística uno contra todos para construir un clasificador de clases múltiples. Dado que hay 10 clases, deberá entrenar 10 clasificadores de regresión logística separados. Para que esta capacitación sea eficiente, es importante asegurarse de que el código esté bien vectorizado.

En esta sección, se implementará una versión vectorizada de regresión logística que no emplea ningún bucle "for".

Para probar la regresión logística vectorizada, se usara datos personalizados como se definen a continuación.

In [874]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [875]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

 en la funcion lrCostFunction(theta, X, y, lambda_)

 se agregado la variable [epsilon]para evitar la division con 0

In [876]:
def lrCostFunction(theta, X, y, lambda_):
 
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size
    epsilon = 1e-8 
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h+epsilon)) - (1 - y).dot(np.log(1 - h+epsilon))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

#### 1.3.3 Vectorización regularizada de la regresión logística




In [877]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


<a id="section2"></a>
### 1.4 Clasificacion One-vs-all


In [878]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 1500}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

<a id="section3"></a>
#### 1.4.1 Prediccion One-vs-all

Después de entrenar el clasificador de one-vs-all, se puede usarlo para predecir el dígito contenido en una imagen determinada. Para cada entrada, debe calcular la "probabilidad" de que pertenezca a cada clase utilizando los clasificadores de regresión logística entrenados. La función de predicción one-vs-all seleccionará la clase para la cual el clasificador de regresión logística correspondiente genera la probabilidad más alta y devolverá la etiqueta de clase (0, 1, ..., K-1) como la predicción para el ejemplo de entrada.

In [879]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Una vez que haya terminado, se llama a la función `predictOneVsAll` usando el valor aprendido de $\theta$. Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 95,1% (es decir, clasifica correctamente el 95,1% de los ejemplos del conjunto de entrenamiento).

In [880]:
lambda_ = 0.002 #mejor lambda es 0.002
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)
print(all_theta)

(6, 10)
[[-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [ 1.89656186e+01  1.93975898e-01 -1.12016860e-01  1.86676605e-01
  -3.50816074e-01 -2.42674154e-01 -1.25760901e-01 -8.02184625e-02
  -4.52411897e-01 -1.09966833e+00]
 [-1.61651377e+00 -4.27264539e-02 -3.63512451e-01  9.03462918e-02
   3.25894494e-02  6.16630311e-02  3.28827025e-03  1.09738519e-01
   9.81226155e-02  2.75455887e+00]
 [ 2.24467555e+01 -6.63300752e-01 -2.02616906e-01  1.19692581e-01
  -2.97595063e-01 -2.87098144e-01  1.94180693e-02 -2.74849196e-02
   1.08209975e-01 -9.01805344e-01]
 [-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [-2.10186903e+01  9.10684499e-01  1.75128729e-01 -1.38587569e-01
   7.06298698e-01  1.69360145e-01  3.97495922e-01  1.41021682e-01
  -6.12505514e-01  1.2320

In [881]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = Xprueba[204:205, :].copy()
y_p=yprueba[204]
print("se espera de clase :",y_p)
XPrueba = np.concatenate([np.ones((1, 1)), XPrueba], axis=1)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print("La clase es: ", p)

#displayData(X[1002:1003, :])

(500, 9)
Precision del conjuto de entrenamiento: 40.00%
se espera de clase : 1.0
La clase es:  [1]
